# Bitcoin price visualiser

In [1]:
import os
import quandl
import pickle
import plotly
import numpy as pd
import pandas as pd

import config

import plotly.offline as py # offline version of plotly 
import plotly.graph_objs as go  # plotly graphical object
py.init_notebook_mode(connected=True)

import chart_studio

In [2]:
quandl.ApiConfig.api_key = config.key
chart_studio.tools.set_credentials_file(username=config.username, api_key=config.cs_key)

## Cache bitcoin data

In [3]:
def get_quandl_data(quandl_code):
    """
    Provide price data for given quandl_code via caching mechanism 
    """
    cache_path = "{}.pkl".format(quandl_code).replace("/", "-")
    
    # load price data if it exists
    try:
        file = open(cache_path, "rb")
        df = pickle.load(file)
        print(f"Loaded {quandl_code} data from cache")
    # othewise cache the data
    except (OSError, IOError):
        df = quandl.get(quandl_code, returns="pandas")
        df.to_pickle(cache_path)
        print(f"Cached {quandl_code} data to {cache_path}")
    return df

In [4]:
btc_usd_kraken = get_quandl_data("BCHARTS/KRAKENUSD")

Loaded BCHARTS/KRAKENUSD data from cache


In [5]:
btc_usd_kraken.head()

,Open,High,Low,Close,Volume (BTC),Volume (Currency),Weighted Price
Date,,,,,,,
2014-01-07,874.67040,892.06753,810.00000,810.00000,15.622378,13151.472844,841.835522
2014-01-08,810.00000,899.84281,788.00000,824.98287,19.182756,16097.329584,839.156269
2014-01-09,825.56345,870.00000,807.42084,841.86934,8.158335,6784.249982,831.572913
2014-01-10,839.99000,857.34056,817.00000,857.33056,8.024510,6780.220188,844.938794
2014-01-11,858.20000,918.05471,857.16554,899.84105,18.748285,16698.566929,890.671709


In [6]:
# setting up traces for plotting
btc_trace = go.Scatter(x=btc_usd_kraken.index, y=btc_usd_kraken["Weighted Price"])
btc_trace

Scatter({
    'x': array([datetime.datetime(2014, 1, 7, 0, 0),
                datetime.datetime(2014, 1, 8, 0, 0),
                datetime.datetime(2014, 1, 9, 0, 0), ...,
                datetime.datetime(2021, 3, 10, 0, 0),
                datetime.datetime(2021, 3, 11, 0, 0),
                datetime.datetime(2021, 3, 12, 0, 0)], dtype=object),
    'y': array([  841.8355223 ,   839.15626937,   831.57291257, ..., 55601.55189567,
                56549.24334555, 57906.23755731])
})

Trace has been setup correctly so we can proceed to plotting the data

In [7]:
py.iplot([btc_trace])

The plot shows that there are days where the bitcoin price data is zero even though this has not occurred in the actual price history. Hence, data from other exchanges will need to be aggregated for more realistic price history.

In [8]:
btc_usd_kraken = get_quandl_data("BCHARTS/KRAKENUSD")

Loaded BCHARTS/KRAKENUSD data from cache


## Get data from multiple exchanges

In [9]:
# popular exchanges
exchanges = ["COINBASE", "OKCOIN", "BITSTAMP", "KRAKEN", "BITFINEX", "ITBIT"]

# collate exchange data
exchange_data = {}

for exchange in exchanges:
    exchange_code = f"BCHARTS/{exchange}USD"
    exchange_data[exchange] = get_quandl_data(exchange_code)

Loaded BCHARTS/COINBASEUSD data from cache
Loaded BCHARTS/OKCOINUSD data from cache
Loaded BCHARTS/BITSTAMPUSD data from cache
Loaded BCHARTS/KRAKENUSD data from cache
Loaded BCHARTS/BITFINEXUSD data from cache
Loaded BCHARTS/ITBITUSD data from cache


In [10]:
def merge_datafs(dataframes, labels, col):
    """
    Make dataframe which includes specified data from each exchanges dataframe
    """
    series_dict = {}
    
    # extract given col from each exchange dataframe
    for index in range(len(labels)):
        series_dict[labels[index]] = dataframes[index][col]
    return pd.DataFrame(series_dict)

In [11]:
btc_usd_df = merge_datafs(list(exchange_data.values()), list(exchange_data.keys()), "Weighted Price")

Most exchanges don't have data for the genesis of bitcoin

In [12]:
btc_usd_df.head()

,COINBASE,OKCOIN,BITSTAMP,KRAKEN,BITFINEX,ITBIT
Date,,,,,,
2011-09-13,NaN,NaN,5.929231,NaN,NaN,NaN
2011-09-14,NaN,NaN,5.590798,NaN,NaN,NaN
2011-09-15,NaN,NaN,5.094272,NaN,NaN,NaN
2011-09-16,NaN,NaN,4.854515,NaN,NaN,NaN
2011-09-17,NaN,NaN,4.870000,NaN,NaN,NaN


But we do have more receent information so we'll take the average of the prices from each exchange which has prices

In [13]:
btc_usd_df.tail()

,COINBASE,OKCOIN,BITSTAMP,KRAKEN,BITFINEX,ITBIT
Date,,,,,,
2021-03-09,NaN,NaN,53979.590696,53898.645831,NaN,NaN
2021-03-10,NaN,NaN,55654.479987,55601.551896,NaN,NaN
2021-03-11,NaN,NaN,56338.627194,56549.243346,NaN,NaN
2021-03-12,NaN,NaN,56711.967124,57906.237557,NaN,NaN
2021-03-13,NaN,NaN,57213.901514,NaN,NaN,NaN


## Visualise aggregate btc price data

In [14]:
layout = go.Layout(
        title="Bitcoin price by exchange",
        legend={"orientation":"h"},
        xaxis={"type":"date"},
        yaxis={"title":"price ($)"})

# setting up traces for plotting
trace_arr = []
labels = list(btc_usd_df)

for _, label in enumerate(labels):
    series = btc_usd_df[label]
    trace = go.Scatter(x=series.index, y=series, name=label)
    trace_arr.append(trace)
    
fig = go.Figure(data=trace_arr, layout=layout)
py.iplot(fig)

Average the data to address price discontinuities across exchanges

In [15]:
btc_usd_df["Average Weighted Price"] = btc_usd_df.mean(axis=1)

In [16]:
layout_avg = go.Layout(
        title="Average bitcoin price",
        legend={"orientation":"h"},
        xaxis={"type":"date"},
        yaxis={"title":"price ($)"})

btc_mean_trace = go.Scatter(x=btc_usd_df.index, 
                            y=btc_usd_df["Average Weighted Price"], name="Average Price")

fig_avg = go.Figure(data=btc_mean_trace, layout=layout_avg)
py.iplot(fig_avg)

## Display chart online

Average bitcoin price

In [18]:
chart_studio.plotly.plot(fig_avg, filename="Average bitcoin price", auto_open=True)

'https://plotly.com/~Azuremis/3/'